## Esercizio

### Consegna

- Scegliere un verbo transitivo --> **KILL**
- Trovare un corpus con > 1000 frasi in cui comprare un verbo scelto (usare un verbo comune) --> link to resource: https://sentence.yourdictionary.com/kill
- Effettuare parsing e disambiguazione
- Usare i supersensi di wordnet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Calcolo risultati, frequenza e stampare cluster semantici ottenuti

### Domande per il prof

- Il modo in ci etichettiamo "person" alle parole va bene?
- Il modo in cui disambiguiamo, per quanto poco efficiente (lesk), è ok?
- Va bene il modo in cui rappresentiamo i risultati (cluster semantici)?

### Appunti

Wordnet supersense: Per ogni sinset abbiamo un supersenso associato (esempi su slide)
Utilizzare i supersense dei synset per determinare il semantic type - nltk

I supersense di wordenet non sono il massimo, vediamo alcune alternative:
 * CSI - Ai supersense ci sono associati delle categorie che possono essere utilizzati come supersense

link to wordnet resource: https://www.nltk.org/_modules/nltk/corpus/reader/wordnet.html

*Spunti futuri:*

**Come distinguo i synset?**
--> Funzione lesk (https://www.nltk.org/howto/wsd.html#word-sense-disambiguation)

**Come trovo il supersenso tra due termini?**
--> Funzione Lowest Common Hypernyms (https://www.nltk.org/howto/wordnet_lch.html)

### Esecuzione

In [55]:
from nltk.corpus import wordnet
from spacy.matcher import DependencyMatcher
from nltk.wsd import lesk
import re
import spacy
from pprint import pprint

* person: Usata per trovare il supersenso dei pronomi
* patter: Usato per trovare il soggetto e l'oggetto del verbo kill

In [56]:
person = ["i", "you", "he", "she", "we", "they", "me", "him", "her", "his", "them", "someone", "us", "people", "anyone"] 

pattern = [
    {"RIGHT_ID": "attr",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["kill"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

#### carico spacy e aggiungo il pattern al Matcher

In [57]:
nlp = spacy.load('en_core_web_sm')
matcher = DependencyMatcher(nlp.vocab)
matcher.add("pattern", [pattern])

#### Metodi utili per trovare il match e per la wsd

In [58]:
def get_match( text):
    # Find the pattern in the document
    doc = nlp(text)
    matches = matcher(doc)
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]
        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text
        
        return subj,dobj,phrase[0].tag_,phrase[len(phrase)-1].tag_
    return "","","",""

def word_sense_disambiguation(list_words, word):
    right_synset = lesk(list_words, word)
    return right_synset

In [59]:
def cleaner(text):
    res = text.split('.')
    return res[1]

In [60]:
subj_ss = ""
dobj_ss = ""
struct = {}
with open ('../sentence_kill.txt', 'r') as f:
    for row in f:
        subj_synset, dobj_synset, subj_ss, dobj_ss = None, None, "", ""
        subj, dobj, stag, dtag = get_match(row)
            
        if subj != "" and dobj != "":
            # Cerco il synset del soggetto e dell'oggetto e associo automaticamente il synset "person" se trovo
            # un nome proprio o una sringa presente in person
            
            # Soggetto
            if stag == "NNP" or subj.lower() in person:
                subj_ss = "person"
            else:
                subj_synset = word_sense_disambiguation(re.findall(r'\w+', row), subj)
            # Oggetto
            if dtag == "NNP" or dobj.lower() in person:
                dobj_ss = "person"
            else:
                dobj_synset = word_sense_disambiguation(re.findall(r'\w+', row), dobj)        
                 
            # Se subj_synset e' None, significa che abbiamo associato il synset person
            if not subj_synset is None: 
                subj_ss = cleaner(subj_synset.lexname())
            elif subj_ss != "person":
                subj_ss = "unknown"
            
            # Soggetto
            if not dobj_synset is None:
                dobj_ss = cleaner(dobj_synset.lexname())    
            elif dobj_ss != "person":
                dobj_ss = "unknown"

            # Oggetto
            if (subj_ss, dobj_ss) in struct:
                struct[(subj_ss, dobj_ss)] += 1
            else:
                struct[(subj_ss, dobj_ss)] = 1
            
for k in struct.keys():
    print(f"{k}: {round(((struct[k]/tot)*100), 2)}%")

{('Tops', 'person'): 1,
 ('act', 'cognition'): 1,
 ('act', 'person'): 2,
 ('all', 'food'): 1,
 ('animal', 'artifact'): 1,
 ('animal', 'quantity'): 1,
 ('artifact', 'act'): 1,
 ('artifact', 'all'): 1,
 ('artifact', 'communication'): 2,
 ('artifact', 'person'): 1,
 ('body', 'person'): 1,
 ('cognition', 'person'): 3,
 ('communication', 'food'): 1,
 ('communication', 'person'): 1,
 ('competition', 'artifact'): 1,
 ('contact', 'person'): 2,
 ('food', 'person'): 1,
 ('food', 'time'): 1,
 ('group', 'consumption'): 1,
 ('group', 'person'): 6,
 ('motion', 'person'): 1,
 ('person', 'act'): 2,
 ('person', 'all'): 8,
 ('person', 'animal'): 6,
 ('person', 'artifact'): 4,
 ('person', 'change'): 1,
 ('person', 'cognition'): 2,
 ('person', 'communication'): 4,
 ('person', 'emotion'): 2,
 ('person', 'food'): 1,
 ('person', 'group'): 3,
 ('person', 'person'): 108,
 ('person', 'stative'): 1,
 ('person', 'unknown'): 10,
 ('social', 'person'): 2,
 ('social', 'state'): 1,
 ('unknown', 'animal'): 2,
 ('unkno

## Vecchio metodo che non etichettava i nomi propri correttamente (per fare comparazione di performance):

In [62]:
def get_match_2( text):
    # Find the pattern in the document
    doc = nlp(text)
    matches = matcher(doc)
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]
        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text
        
        return subj,dobj
    return "",""

subj_ss = ""
dobj_ss = ""
struct = {}
with open ('../sentence_kill.txt', 'r') as f:
    for row in f:
        subj_ss, dobj_ss = "", ""
        subj, dobj = get_match_2(row)
        if subj != "" and dobj != "":
            subj_synset = word_sense_disambiguation(re.findall(r'\w+', row), subj)
            dobj_synset = word_sense_disambiguation(re.findall(r'\w+', row), dobj)
            
            if not subj_synset is None: 
                subj_ss = cleaner(subj_synset.lexname())
            else:
                if subj.lower() in person:
                    subj_ss = "person"
                else:
                    subj_ss = subj.lower()
            if not dobj_synset is None:
                dobj_ss = cleaner(dobj_synset.lexname())    
            else:
                if dobj.lower() in person:
                    dobj_ss = "person"
                else:
                    dobj_ss = dobj.lower()

            if (subj_ss, dobj_ss) in struct:
                struct[(subj_ss, dobj_ss)] += 1
            else:
                struct[(subj_ss, dobj_ss)] = 1
            
pprint(struct)

{('Tops', 'animal'): 1,
 ('act', 'cognition'): 1,
 ('act', 'person'): 3,
 ('all', 'all'): 2,
 ('all', 'animal'): 2,
 ('all', 'artifact'): 1,
 ('all', 'change'): 1,
 ('all', 'cognition'): 1,
 ('all', 'communication'): 1,
 ('all', 'food'): 1,
 ('all', 'group'): 1,
 ('all', 'location'): 1,
 ('all', 'person'): 14,
 ('animal', 'Tops'): 1,
 ('animal', 'artifact'): 1,
 ('animal', 'himself'): 1,
 ('animal', 'location'): 1,
 ('animal', 'person'): 3,
 ('animal', 'quantity'): 1,
 ('artifact', 'act'): 1,
 ('artifact', 'all'): 1,
 ('artifact', 'communication'): 2,
 ('artifact', 'person'): 1,
 ('body', 'person'): 1,
 ('cognition', 'location'): 1,
 ('cognition', 'person'): 2,
 ('communication', 'food'): 1,
 ('communication', 'person'): 1,
 ('competition', 'artifact'): 1,
 ('contact', 'person'): 2,
 ('czerno', 'location'): 1,
 ('food', 'person'): 1,
 ('food', 'time'): 1,
 ('group', 'animal'): 1,
 ('group', 'consumption'): 1,
 ('group', 'person'): 4,
 ('group', 'stative'): 1,
 ('jonny', 'person'): 2,
 